In [1]:
%load_ext autoreload
%autoreload 2

from sympy import *
import pandas as pd

import sys
sys.path.append("../")
from bargains_linear import * 

In [2]:
#general purpose code for interior cases

phi1 =Symbol('\\phi_1') #phi_2
phi2 =Symbol('\\phi_2') #phi_2
p1=Symbol('p_1') #p1 #TODO: integrate this into the formula*****

cost =Symbol('\\lambda') #can't name variable lambda
out = Symbol('\\pi') #outside option: pi_h(g\i)
beta1  = Symbol('\\beta1')
beta2  = Symbol('\\beta2')
mc1 =Symbol('c_1') 
mc2=Symbol('c_2')

In [3]:
def calc_price(phi1, phi2, cost ,mc1,mc2):
    """compute each insurer's price (assumes interior solution)"""
    p1 = (2*(phi1+mc1)+phi2+mc2)/3. + cost
    p2 = (2*(phi2+mc2)+phi1+mc1)/3. + cost
    return p1, p2
  
    
def calc_s(phi1, phi2, cost,  mc1,mc2):
    """compute the proportion of people choosing each insurer
    assumes interior solution"""
    p1, p2 = calc_price(phi1, phi2, cost, mc1,mc2)
    t_hat = .5 + (p2 - p1)/(2.*cost)
    return t_hat


def calc_profits(phi1,phi2,cost,  mc1,mc2):
    s = calc_s(phi1, phi2, cost,  mc1,mc2)
    p1,p2 = calc_price(phi1, phi2, cost, mc1,mc2)

    profits1, profits2 = s*(p1-phi1-mc1), (1-s)*(p2-phi2-mc2)
    hosp_profit = s*(phi1) +  (1-s)*(phi2)

    return hosp_profit, profits1, profits2



#arbitrary outside option...
def nash_in_nash2(phi1, phi2, cost, mc1,mc2,beta2,out):
    hosp_profit, profits1, profits2 = calc_profits(phi1, phi2,  cost,  mc1,mc2)
    return ((hosp_profit-out)**(1-beta2))*(profits2**beta2)



In [4]:
def solve_c(phi1, phi2, cost, mc1,mc2,beta2,out):
    foc =  diff(nash_in_nash2(phi1, phi2, cost, mc1,mc2,beta2,out), phi2)
    foc = foc.subs( {out:phi1} )
    h =solve(foc,mc2)[0]#.subs({mc1:0})
    return h

result1 = solve_c(phi1, phi2, cost, mc1,mc2,beta2,out)
print(result1)

(3.0*\beta2*\lambda + \beta2*c_1 - 3.0*\lambda - 2.0*\phi_1 + 2.0*\phi_2 - c_1)/(\beta2 - 1.0)


In [5]:
reimb1,reimb2 = seq_bargain(25, 5,25,np.array([1,1.5]),betas=[.5,.5])
reimb1,reimb2 = reimb1[0],reimb2[0]
price1,price2 = calc_price(reimb1,reimb2,5,1,1)
print(reimb1,reimb2,price1,price2)
#print(result1[0].subs({beta1:.5,beta2:.5,lamb:5}))
print(result1.subs({beta1:.5,beta2:.5,cost:5,phi1:reimb1,phi2:reimb2,p1:price1}).evalf())

15.937647528656367 19.56261840174708 23.14597115301994 24.35429477738351
1.0*c_1 + 0.500116507637145


In [6]:
def solve_beta2(phi1, phi2, cost, mc1,mc2,beta2,out):
    foc =  diff(nash_in_nash2(phi1, phi2, cost, mc1,mc2,beta2,out), phi2)
    foc = foc.subs( {out:phi1} ) #.subs({mc1:0})
    h =solve(foc,beta2)[0]
    return h

result2 = solve_beta2(phi1, phi2, cost, mc1,mc2,beta2,out)
print(result2)
print(result2.subs({mc2:.5,cost:5,phi1:reimb1,phi2:reimb2,p1:price1}))

(3.0*\lambda + 2.0*\phi_1 - 2.0*\phi_2 + c_1 - c_2)/(3.0*\lambda + c_1 - c_2)
(c_1 + 7.25005825381857)/(c_1 + 14.5)
